# Blog Model

The Blog application demonstrates a Jinaga model expressed in several different forms.
As a Maui application, it showcases content creation and management on a mobile device.
As a React application, it demonstrates reader interaction through an in-browser commenting experience.
And as a function app, it reveals how backend services can respond to the model to perform tasks such as static site generation.

The Blog app is at once familiar and full of expressive possibilities.
This document describes the model while also teaching various features and patterns of Jinaga.

In [ ]:
// Reference the Jinaga NuGet packages
#r "nuget: Jinaga, 0.11.18"
#r "nuget: Jinaga.Graphviz, 0.11.18"
#r "nuget: Jinaga.UnitTest, 0.11.18"

In [ ]:
using Jinaga;
using Jinaga.Graphviz;
using Jinaga.UnitTest;

// Create a Jinaga client for unit testing. Simulate a logged in user.
var jinagaClient = JinagaTest.Create(opt =>
{
    opt.User = new User("--- FAKE USER ---");
});

## Declaring Facts

Express a Jinaga model as a set of C# records representing historical facts.
Decorate each record with a `FactType` attribute.
The fields of the records are predecessors -- facts that came before -- and fields.

In the Blog model, a site is an entity created by a specific user.
We use the date and time to distinguish it from other sites that the user created.
The assumption is that a user will not create multiple sites within the same millisecond.

Notice that the site does not contain any other properties, such as a title or a domain name.
Those properties are mutable, and therefore not part of the initial fact.

In [ ]:
[FactType("Blog.Site")]
public record Site(User creator, DateTime createdAt) { }

The `User` type is provided by the Jinaga library.
It represents someone who can log into the app.
A model almost always starts with a fact owned by a user.
This gives us a place to start our authorization rules.

In [ ]:
// Call RenderTypes without a semicolon to display the graph
Renderer.RenderTypes(typeof(User), typeof(Site))

## Log In

The user is the starting point of the model.
You'll need to call `Login` to get the logged in user.

In [ ]:
// Login returns the user fact and profile information
var (user, profile) = await jinagaClient.Login();

// Call RenderFacts on the Jinaga client to display the facts
jinagaClient.RenderFacts(user)

## Save Facts

Save an instance of a fact to the Jinaga client whenever the user takes an action.
Pass a record to the `Fact` method.
This will save the fact in the local store and share it with the server.

The `Fact` method returns the record that was just saved.
Hold on to this copy of the record for use in other methods.

In [ ]:
var site = await jinagaClient.Fact(new Site(user, DateTime.UtcNow));

jinagaClient.RenderFacts(site)

## Query for Successors

Given one fact, you can find its successors using LINQ.
For example, you can find all sites for a given user.
Use `Given<T>.Match()` to define a specification.

In [ ]:
// The parameters to the lambda are the given fact (in this case, the user) and
// the fact repository. Use the OfType<T> method to get facts from the repository.
var sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    select site);

// Query for facts matching the specification.
var sites = await jinagaClient.Query(sitesByUser, user);

jinagaClient.RenderFacts(sites)